# Mission to Mars

Build a web application that scrapes various websites for data related to the Mission to Mars and display the information in a single HTML page. 

## Step 1 - Scraping

In [1]:
# Dependencies
# Web scraping dependencies
from bs4 import BeautifulSoup

# Asynchronous scraping dependencies
import time
from splinter import Browser
import requests


# Database deployment dependencies
import pymongo
# Data Dependencies
import pandas as pd
import json

In [2]:
# Chromedriver set-up
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Retrieve page with the requests module
MarsNews='https://mars.nasa.gov/news/'
browser.visit(MarsNews)
time.sleep(10)

In [4]:
# Scrape the html on the site
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
browser.quit()

### Nasa Mars News

In [5]:
#find divs
article = soup.find("div", class_='list_text')

#collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later
news_title = article.find("div", class_="content_title").text
news_p = article.find("div", class_ ="article_teaser_body").text

In [6]:
news_title

"Things Are Stacking up for NASA's Mars 2020 Spacecraft"

In [7]:
news_p

'As the July 2020 launch date inches closer, the next spacecraft headed to the Red Planet is assembled for more testing.'

### JPL Space Images Featured Image

In [8]:
# Chromedriver set-up
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [9]:
#Use splinter to navigate the site and find the image url for the current Featured Mars Image 
nasafeature= 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(nasafeature)
time.sleep(10)
html = browser.html
soup1 = BeautifulSoup(html, 'html.parser')

image = soup1.find("article", class_="carousel_item")
image_url = "https://www.jpl.nasa.gov" + image["style"][23:75]
image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18049-1920x1200.jpg'

In [10]:
browser.quit()

### Mars Weather

In [11]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [12]:
#scrape mars weather Twitter account
twitter = "https://twitter.com/marswxreport?lang=en"
response = requests.get(twitter)
soup2 = BeautifulSoup(response.text, "html.parser")
# print(soup2.prettify())

mweather = soup2.find("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
mweather = mweather.replace("\n", " ")
mweather


'InSight sol 141 (2019-04-20) low -98.3ºC (-144.9ºF) high -19.7ºC (-3.5ºF) winds from the SW at 4.7 m/s (10.6 mph) gusting to 12.9 m/s (28.8 mph) pressure at 7.40 hPapic.twitter.com/CQr1QQt3cM'

In [13]:
browser.quit()

### Mars Facts

In [14]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [15]:
#scrape mars facts from mars facts website
marsFacts = "https://space-facts.com/mars/"
tables = pd.read_html(marsFacts , encoding= "utf-8")
type(tables)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [16]:
#DF first table and rename column headers
df = tables[0]
df.columns = ['Mars Facts','Answered']
df.head()

,Mars Facts,Answered
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"


In [17]:
#dump DF into json
df1= json.dumps(df.to_dict(orient='list'))
df1

'{"Mars Facts": ["Equatorial Diameter:", "Polar Diameter:", "Mass:", "Moons:", "Orbit Distance:", "Orbit Period:", "Surface Temperature:", "First Record:", "Recorded By:"], "Answered": ["6,792 km", "6,752 km", "6.42 x 10^23 kg (10.7% Earth)", "2 (Phobos & Deimos)", "227,943,824 km (1.52 AU)", "687 days (1.9 years)", "-153 to 20 \\u00b0C", "2nd millennium BC", "Egyptian astronomers"]}'

In [18]:
browser.quit()

### Mars Hemispheres 

In [19]:
# Chromedriver set-up
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [44]:
hemisphere='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemisphere)


In [46]:
hemis=[]

links= browser.find_by_css("a.product-item h3")
for i in range(len(links)):
#     hemis= {}
    #find elements on each loop to avoida a stale exception
    browser.find_by_css("img.thumb")[i].click()
    time.sleep(5)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    downloads = soup.find("div", class_="downloads")
    image_url = downloads.find("a")["href"]
    title= browser.find_by_css('h2').text
    title= title.replace("Enhanced", "")
    hemis.append({"title": title, "link": image_url})
    browser.back()

In [47]:
hemis

[{'title': 'Cerberus Hemisphere ',
  'link': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere ',
  'link': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere ',
  'link': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere ',
  'link': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

## Step 2 - MongoDB and Flask Application